In [1]:
import re
import urllib.request
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image

# 打开txt文件
with open('stream-response.txt', 'r') as file:
    content = file.read()

# 使用正则表达式提取商品信息
# pattern = r'<h4 class="fs18 black" style="font-size: 12px;">(.+?)<\/h4>'
# matches = re.findall(pattern, content)

In [2]:
brand_txt = '<h4 class="fs18 black" style="font-size: 12px;">(.+?)<\/h4>'
level_txt = '<h5 class="fs12 red">(.+?)<\/h5>'
start_jap = '<h5 class="fs12" style="color: #333;">(.+?)<strong>(.+?)<\/strong>'
# start_jap = '<h5 class="fs12" style="color: #333;">(\d+)<strong>(.+?)<\/strong>'
# start_chi = 
finish_jap = '<p class="fs14 red">(.+?)<strong>(.+?)<\/strong>'
# finish_chi = 
image_txt = '<img src="([^"]+)"'
pattern = r'{}[\s\S]+?{}[\s\S]+?{}[\s\S]+?{}[\s\S]+?{}'.format(brand_txt,level_txt,start_jap,finish_jap,image_txt)

In [3]:
# 使用正则表达式提取商品信息
matches = re.findall(pattern, content)
# matches
df = pd.DataFrame(matches, columns=['品牌', '级别', '起拍价日元','起拍价人民币', '结拍价日元','人民币', '图片链接'])
df

,品牌,级别,起拍价日元,起拍价人民币,结拍价日元,人民币,图片链接
0,ＬＯＵＩＳ ＶＵＩＴＴＯＮ,B级,起拍价：17000≈￥,881,结拍价：21500≈￥,1114,https://image.brand-auc.com/brand_img/0/SP/680...
1,ＬＯＵＩＳ ＶＵＩＴＴＯＮ,B级,起拍价：14000≈￥,726,结拍价：30000≈￥,1554,https://image.brand-auc.com/brand_img/0/SP/680...
2,ＬＯＵＩＳ ＶＵＩＴＴＯＮ,C级,起拍价：5000≈￥,259,结拍价：16500≈￥,855,https://image.brand-auc.com/brand_img/0/SP/678...
3,ＶＥＲＳＡＣＥ,C级,起拍价：5000≈￥,259,结拍价：6000≈￥,311,https://image.brand-auc.com/brand_img/0/SP/679...
4,ＨＥＲＭＥＳ,C级,起拍价：8000≈￥,415,结拍价：25500≈￥,1321,https://image.brand-auc.com/brand_img/0/SP/680...
5,Ｃａｒｔｉｅｒ,B级,起拍价：11000≈￥,570,结拍价：22000≈￥,1140,https://image.brand-auc.com/brand_img/0/SP/680...
6,ｊｅｗｅｌｒｙ（Ｎｏ Ｂｒａｎｄ）,C级,起拍价：5000≈￥,259,结拍价：14000≈￥,726,https://image.brand-auc.com/brand_img/0/SP/680...
7,ＨＥＲＭＥＳ,C级,起拍价：5000≈￥,259,结拍价：7500≈￥,389,https://image.brand-auc.com/brand_img/0/SP/680...
8,ＣＨＡＮＥＬ,D级,起拍价：38000≈￥,1969,结拍价：54000≈￥,2798,https://image.brand-auc.com/brand_img/0/SP/680...


In [4]:

df.to_excel('商品信息.xlsx', index=False)


# 创建一个新的 Excel 工作簿
workbook = Workbook()
sheet = workbook.active

# 设置每行的行高为 30
for i in range(1, len(matches) + 2):
    sheet.row_dimensions[i].height = 50
    
# 设置每列的宽度
column_widths = [15, 15, 15, 15, 15, 15, 30] # 前几个数字是文字列宽，最后一个是图片列宽
for i, width in enumerate(column_widths):
    column_letter = get_column_letter(i + 1)
    sheet.column_dimensions[column_letter].width = width
    
    
# 插入商品信息到工作表
for i, match in enumerate(matches):
    brand, level, start_price, start_2,end_price,end_2, image_url = match

    # 下载图片并保存到本地
    image_path = f'image_{i + 1}.jpg'
    urllib.request.urlretrieve(image_url, image_path)

    # 在工作表中插入图片
    img = Image(image_path)
    # 调整图片大小以适应单元格
#     img_width = sheet.column_dimensions['E'].width - 2  # 图片宽度为单元格宽度减去一些间距
    img_width = 50  # 图片的宽度
    img_height = img.height * (img_width / img.width)
    img.width = img_width
    img.height = img_height
    sheet.add_image(img, f'G{i+2}')
    
    # 插入其他字段值
    sheet[f'A{i+2}'] = brand
    sheet[f'B{i+2}'] = level
    sheet[f'C{i+2}'] = start_price
    sheet[f'D{i+2}'] = start_2
    sheet[f'E{i+2}'] = end_price
    sheet[f'F{i+2}'] = end_2

# 保存 Excel 文件
workbook.save('商品信息.xlsx')

print("商品信息处理完成并保存成功！")



商品信息处理完成并保存成功！
